# Testing with multiple Masks

In [8]:
# Iting packages 
import os
import numpy as np
import nilearn
import glob
import nibabel as nib
import pandas as pd 
from sklearn.model_selection import cross_val_score
from nilearn.input_data import NiftiMasker 
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler
import numpy as np
import warnings
import matplotlib
import multiprocessing
from multiprocessing import Pool
import time
from sklearn import preprocessing



np.seterr(divide='ignore', invalid='ignore')
warnings.filterwarnings("ignore")
matplotlib.use('Agg')  
    #mask='/projects/niblab/bids_projects/Experiments/ChocoData/images/bin_mask.nii.gz'
mask='/Users/nikkibytes/Documents/lab/test_imgs/bin_mask.nii.gz'
    #our behavioral csv file 
    #stim = '/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/all_waves_b1.csv'
stim='/Users/nikkibytes/Documents/lab/test_imgs/sub-001.csv'
    #our dataset concatenated image 
    #dataset='/projects/niblab/bids_projects/Experiments/ChocoData/images/all_waves_b1.nii.gz'
dataset='/Users/nikkibytes/Documents/lab/test_imgs/sub-001.nii.gz'
    
    #load behavioral data into a pandas df
labels = pd.read_csv(stim, sep="\t")


# load labels
stimuli = labels['Label']
# identify resting state labels in order to be able to remove them
task_mask = (stimuli != 'rest')

# find names of remaining active labels
categories = stimuli[task_mask].unique()

# extract tags indicating to which acquisition run a tag belongs
session_labels = labels["sess"][task_mask]
#print(session_labels)                       
                        

In [ ]:
from sklearn.svm import SVC
classifier = SVC(C=1., kernel="linear")

# A classifier to set the chance level
from sklearn.dummy import DummyClassifier
dummy_classifier = DummyClassifier()

# Make a data splitting object for cross validation
from sklearn.model_selection import LeaveOneGroupOut, cross_val_score
cv = LeaveOneGroupOut()

mask_names = ['mask_vt', 'mask_face', 'mask_house']

mask_scores = {}
mask_chance_scores = {}

for mask_name in mask_names:
    print("Working on mask %s" % mask_name)
    # For decoding, standardizing is often very important
    mask_filename = haxby_dataset[mask_name][0]
    masker = NiftiMasker(mask_img=mask_filename, standardize=True)
    masked_timecourses = masker.fit_transform(
        func_filename)[task_mask]

    mask_scores[mask_name] = {}
    mask_chance_scores[mask_name] = {}

    for category in categories:
        print("Processing %s %s" % (mask_name, category))
        classification_target = (stimuli[task_mask] == category)
        mask_scores[mask_name][category] = cross_val_score(
            classifier,
            masked_timecourses,
            classification_target,
            cv=cv,
            groups=session_labels,
            scoring="roc_auc",
        )

        mask_chance_scores[mask_name][category] = cross_val_score(
            dummy_classifier,
            masked_timecourses,
            classification_target,
            cv=cv,
            groups=session_labels,
            scoring="roc_auc",
        )

        print("Scores: %1.2f +- %1.2f" % (
            mask_scores[mask_name][category].mean(),
            mask_scores[mask_name][category].std()))